# LangGraph with Claude API Demo

This notebook demonstrates a basic LangGraph application that interacts with the Claude API.

In [ ]:
import os
from dotenv import load_dotenv
from agent import create_agent

# Load environment variables
load_dotenv()

# Configuration
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
MCP_SERVER_URL = "http://0.0.0.0:3000/mcp"
MODEL_ID = "claude-sonnet-4-20250514"

# Create the agent
agent = create_agent(ANTHROPIC_API_KEY, MCP_SERVER_URL, MODEL_ID)

In [ ]:
# Test the agent with memory functionality
print("Testing MCP-enabled Claude agent with memory...")

# Test 1: Initial restaurant search
print("\n" + "="*60)
print("Test 1: Initial restaurant search")
print("="*60)
response1 = agent.chat("I want to find Italian restaurants in Taipei for a romantic dinner date tonight.")
print(f"Claude: {response1}")

# Test 2: Follow-up question (should remember context)
print("\n" + "="*60)
print("Test 2: Follow-up question (should remember context)")
print("="*60)
response2 = agent.chat("What about the second restaurant you mentioned?")
print(f"Claude: {response2}")

# Test 3: Show conversation history
print("\n" + "="*60)
print("Test 3: Conversation history")
print("="*60)
agent.show_conversation_history()

In [ ]:
# Interactive chat session
print("Starting interactive chat session...")
print("You can now chat with Claude using the MCP restaurant booking tools!")
print("The agent will remember your conversation across multiple exchanges.")
print()

# Uncomment the line below to start interactive chat
# agent.interactive_chat()

# Or show the demo
agent.demo_tools()

In [ ]:
# Utility functions for testing different scenarios

def test_multi_turn_conversation():
    """Test a multi-turn conversation with different thread IDs"""
    print("Testing multi-turn conversation scenarios...")
    
    # Scenario 1: Restaurant search and follow-up
    thread1 = "scenario_1"
    print(f"\n--- Scenario 1 (Thread: {thread1}) ---")
    
    response1 = agent.chat("Find me casual restaurants near Ximending in Taipei", thread1)
    print(f"User: Find me casual restaurants near Ximending in Taipei")
    print(f"Claude: {response1[:200]}...")
    
    response2 = agent.chat("What are the opening hours for the first restaurant?", thread1)
    print(f"\nUser: What are the opening hours for the first restaurant?")
    print(f"Claude: {response2[:200]}...")
    
    # Scenario 2: Different conversation
    thread2 = "scenario_2"
    print(f"\n--- Scenario 2 (Thread: {thread2}) ---")
    
    response3 = agent.chat("I need a business lunch venue for 6 people", thread2)
    print(f"User: I need a business lunch venue for 6 people")
    print(f"Claude: {response3[:200]}...")
    
    # Show both conversation histories
    print(f"\n--- History for {thread1} ---")
    agent.show_conversation_history(thread1)
    
    print(f"\n--- History for {thread2} ---")
    agent.show_conversation_history(thread2)

def test_tool_usage():
    """Test specific MCP tool usage scenarios"""
    print("Testing MCP tool usage scenarios...")
    
    scenarios = [
        "Search for romantic Italian restaurants in Taipei for 2 people",
        "I want to check availability at a restaurant for tomorrow at 7 PM",
        "Help me make a reservation for 4 people at 8 PM on Friday"
    ]
    
    thread_id = "tool_test"
    for i, scenario in enumerate(scenarios, 1):
        print(f"\n--- Tool Test {i} ---")
        print(f"User: {scenario}")
        response = agent.chat(scenario, thread_id)
        print(f"Claude: {response[:300]}...")

# Run the utility functions
print("Available test functions:")
print("- test_multi_turn_conversation(): Test multiple conversation threads")
print("- test_tool_usage(): Test MCP tool usage scenarios")
print("\nCall these functions to run specific tests!")

In [ ]:
# Define initialization and processing functions
def initialize_mcp_session(state: State) -> State:
    """Initialize MCP session and load available tools - only run once"""
    # Check if MCP is already initialized
    if state.get("mcp_session_id") and state.get("available_tools"):
        print("MCP already initialized, skipping...")
        return state
    
    success = mcp_client.initialize()
    
    if success:
        print(f"MCP session initialized successfully!")
        print(f"Available tools: {list(mcp_client.tools.keys())}")
        return {
            "mcp_session_id": mcp_client.session_id or "",
            "available_tools": mcp_client.tools,
            "messages": state.get("messages", [])  # Preserve existing messages
        }
    else:
        print("Failed to initialize MCP session")
        return {
            "mcp_session_id": "",
            "available_tools": {},
            "messages": state.get("messages", [])  # Preserve existing messages
        }

def process_input(state: State) -> State:
    """Process the user input before sending to Claude"""
    user_input = state["user_input"]
    
    # Add some basic preprocessing
    processed_input = user_input.strip()
    
    # Preserve all existing state and only update the processed input
    return {
        **state,  # Keep all existing state
        "user_input": processed_input
    }

In [ ]:
# Enhanced chat function with memory support
def run_claude_chat_with_mcp(user_input: str, thread_id: str = "default_conversation"):
    """Run the LangGraph application with user input and MCP tools with memory"""
    config = {"configurable": {"thread_id": thread_id}}
    
    # Get current conversation state from memory
    try:
        current_state = app.get_state(config)
        if current_state.values:
            # Continue existing conversation
            print(f"Debug - Continuing conversation with state keys: {list(current_state.values.keys())}")
            input_state = {
                **current_state.values,
                "user_input": user_input
            }
        else:
            # Start new conversation
            print("Debug - Starting new conversation")
            input_state = {
                "messages": [],
                "user_input": user_input,
                "claude_response": "",
                "mcp_session_id": "",
                "available_tools": {}
            }
    except Exception as e:
        print(f"Debug - Exception getting state: {e}")
        # Fallback for new conversation
        input_state = {
            "messages": [],
            "user_input": user_input,
            "claude_response": "",
            "mcp_session_id": "",
            "available_tools": {}
        }
    
    # Debug: Show message structure before processing
    existing_messages = input_state.get("messages", [])
    print(f"Debug - Existing messages count: {len(existing_messages)}")
    if existing_messages:
        print(f"Debug - First few messages: {existing_messages[:2]}")
    
    result = app.invoke(input_state, config)
    return result["claude_response"]

# Test the enhanced application with memory - start fresh
print("Testing MCP-enabled Claude agent with memory...")
print("First, let's test with a fresh conversation thread...")

# Use a new thread ID to start clean
test_thread_id = "test_memory_v2"
response1 = run_claude_chat_with_mcp("I want to find Italian restaurants in Taipei for a romantic dinner date tonight.", test_thread_id)
print("\nClaude's first response:")
print(response1)

print("\n" + "="*50)
print("Testing follow-up question (should remember context)...")
response2 = run_claude_chat_with_mcp("What about the second restaurant you mentioned?", test_thread_id)
print("\nClaude's follow-up response:")
print(response2)

In [ ]:
config = {"configurable": {"thread_id": test_thread_id}}
current_state = app.get_state(config)

In [ ]:
# Interactive chat function with memory support
def interactive_chat_with_mcp(thread_id: str = "interactive_session"):
    """Interactive chat with Claude via LangGraph and MCP tools with memory"""
    print("Chat with Claude (powered by MCP restaurant booking tools)!")
    print("Memory-enabled: Claude will remember our conversation!")
    print("Available commands:")
    print("- Ask for restaurant recommendations")
    print("- Request restaurant details") 
    print("- Check availability and make reservations")
    print("- Type 'quit' to exit")
    print("- Type 'reset' to start a new conversation")
    print("-" * 50)
    
    current_thread_id = thread_id
    
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye!")
            break
        elif user_input.lower() == 'reset':
            import uuid
            current_thread_id = f"session_{str(uuid.uuid4())[:8]}"
            print(f"Started new conversation (thread: {current_thread_id})")
            continue
        
        try:
            # Use memory-enabled chat function
            response = run_claude_chat_with_mcp(user_input, current_thread_id)
            print(f"\nClaude: {response}")
        except Exception as e:
            print(f"\nError: {e}")

# Function to show conversation history
def show_conversation_history(thread_id: str = "default_conversation"):
    """Display the conversation history for a thread"""
    config = {"configurable": {"thread_id": thread_id}}
    try:
        state = app.get_state(config)
        if state.values and "messages" in state.values:
            messages = state.values["messages"]
            print(f"\nConversation History (Thread: {thread_id}):")
            print("=" * 50)
            for i, msg in enumerate(messages, 1):
                role = msg.get("role", "unknown")
                content = msg.get("content", "")
                if isinstance(content, str):
                    print(f"{i}. {role.upper()}: {content[:200]}{'...' if len(content) > 200 else ''}")
                else:
                    print(f"{i}. {role.upper()}: [Complex content]")
            print("=" * 50)
        else:
            print(f"No conversation history found for thread: {thread_id}")
    except Exception as e:
        print(f"Error retrieving history: {e}")

# Example tool usage demonstrations
def demo_mcp_tools():
    """Demonstrate the MCP tools functionality with memory"""
    print("MCP Tools Demo with Memory")
    print("=" * 50)
    
    examples = [
        "Find me romantic Italian restaurants in Taipei",
        "Tell me more about the first restaurant",
        "What about availability for tomorrow at 7pm?",
        "Can you help me make a reservation?"
    ]
    
    print("Try these example conversations to see memory in action:")
    for i, example in enumerate(examples, 1):
        print(f"{i}. {example}")
    
    print("\nRun interactive_chat_with_mcp() to start chatting!")
    print("Use show_conversation_history() to see saved conversations!")

# Run the demo
demo_mcp_tools()